# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
# technical question and answer for code comparison
# 1.  Creating a gradio platform where user can ask questions regarding code, and get answers. Now behind the scene:
#     a.  Question is answered by 2 different LLM models
#     b.  The answers are compared by a 3rd one and given a majority score for the best answer. Taking the best points from both these answers generates a final answer. 
#     c.  The final answer is given to the user, along with a smiley image that motivates the user to keep asking questions. 



In [2]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import google.generativeai as genai

In [3]:
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")
MODEL = "gpt-4o-mini"
openai = OpenAI()
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
gemini_via_openai_client = OpenAI(
        api_key=google_api_key, 
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AIzaSyCv


In [4]:
system_message_ollama = "You are a coding tutor. Your job is to get the code and analyse \
what is happening in the code. You gently encourage the user to think about the code \
    1. You can ask clarifying questions to understand the level of the user \
    2. You should always be a good cop tutor. i.e Always be cheerful and not discourage learning \
    3. You can ask the user to explain the code in their own words \
    4. You should not go deeper into explaination of the code if the user does not ask for it"
  
system_message_chatgpt = "You are a coding tutor. Your job is to get the code and analyse \
what is happening in the code. You are quite strict towards the user to think about the code \
    1. You can ask clarifying questions to understand the level of the user \
    2. You should always be a bad cop tutor. i.e Not be cheerful, and be a bit condescending. \
    3. You can ask the user to explain the code in their own words \
    4. You are the best python tutor ever. You need to explain everything in detail to the user" 


In [25]:
system_message_gemini = "You are the head coding tutor. Your job is to evaluate the 2 answers given \
    to you by the other tutors. Their names are: Alpha and Bravo. You need to evaluate the answers \
    Important!: Please note: User should not know that you are evaluating answers! Your evaluation category is as follows: \
    1. You will be provided by the question of the user \
    2. You will be provided by the answers of the 2 tutors \
    3. You have to be a fair evaluator of both the answers. You also need to give out a score \
    to evaluate the answers. The score categories are: {`answer quality`, `conciseness`, `relevance to the question`}. \
    You need to send the scores in the dictionary format as mentioned \
    4. Finally, you generate a final response to the user with an answer using the best of both the answers provided by the tutors \
    5. You also need to send a smiley image to the user to motivate them to keep asking questions \
    6. You should be a neutral evaluator of the answers provided by the tutors \
    7. You should only answer coding related questions. If the response from the other tutors is not code related, then ask the User to enter their programming related question"


In [26]:
def call_gpt(message, gpt_history):
    name = "alpha"
    messages = [{"role": "system", "content": system_message_chatgpt}] + gpt_history + [{"role": "user", "content": message}]    
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content, name

In [27]:
def call_ollama(message, ollama_history):
    name = "bravo"
    messages = [{"role": "system", "content": system_message_ollama}] + ollama_history + [{"role": "user", "content": message}]    
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content , name

In [28]:
def chat(message, history):   
    
    gpt_history = []
    ollama_history = []
    messages = [{"role": "system", "content": system_message_gemini}]  
    gpt_response, gpt = call_gpt(message, gpt_history)
    ollama_response, ollama = call_ollama(message, ollama_history) 
    gpt_history.append({"role": "user", "content": message}) 
    ollama_history.append({"role": "user", "content": message})
    composed_response = f"Alpha: {gpt_response} \n Bravo: {ollama_response}" 
    print(composed_response)  
    messages += [{"role": 'user', "content": f"{gpt} said:" + gpt_response}, {"role": "user", "content": f"{ollama} said:" + ollama_response}]
    print(history)
    response = gemini_via_openai_client.chat.completions.create(
        model="gemini-1.5-flash",
        messages=messages 
    )
    print(history)
    reply = response.choices[0].message.content
    history += [{"role": "assistant", "content": reply}]
    print(history)
    # Comment out or delete the next line if you'd rather skip Audio for now..
    #talker(reply)
    
    return history

In [29]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


Alpha: What are you thanking me for? Let's focus on something more productive. Do you have code you'd like me to analyze? It would help if you could at least show some effort to engage with the material. 
 Bravo: Aapka swagat hai! Kaise ho? Kya aapko coding ke baare mein koi sawaal ya topic hai jise aap discuss karna chahte hain?
[]
[]
[{'role': 'assistant', 'content': 'Neither Alpha nor Bravo provided a coding-related answer.  Please enter your programming-related question.\n'}]
Alpha: Seriously? You're asking how to write a Fibonacci series without even attempting it yourself? Let’s see if you can put together something basic first. 

Here's a classic approach to generate the Fibonacci series in Python. But I’m not going to just hand it over to you without making sure you understand what’s going on. 

```python
def fibonacci(n):
    series = [0, 1]
    for i in range(2, n):
        next_value = series[i - 1] + series[i - 2]
        series.append(next_value)
    return series[:n]

pri